In [57]:
import psycopg2,datetime
from psycopg2 import Error
'''CREATE TABLE memorandums
    (ID SERIAL PRIMARY KEY,
    userID TEXT NOT NULL,
    DATE DATE NOT NULL,
    TODO TEXT NOT NULL); 
    '''
    #connection.commit()
def connect(SQLcode,*variables):
    try:
        connection = psycopg2.connect(user="refalaxzgyswai",
                                      password="",
                                      host="",
                                      port="",
                                      database="")  
        cursor = connection.cursor()
        cursor.execute(SQLcode,tuple(variables))
        connection.commit()
        if cursor.description!=None:
            res=cursor.fetchall()
            print("抓資料成功")
            return res
    except (Exception, Error) as error:
        print(error)
    finally:
        if (connection):
            cursor.close()
            connection.close()
            print("連線關閉")
def allMemos(userId):
    res=''
    for memo in connect('SELECT * FROM memorandums WHERE userID=%s;',userId):
            res+=memo[1]+memo[2].strftime('%Y/%m/%d ')+memo[3]+'\n'
    return res
def newMemo(userId,date,toDo):
    connect('INSERT INTO memorandums(userID,DATE,TODO) VALUES (%s,%s,%s);',userId,date,toDo)
    return '完成'
def deleteMemo(userId,date,toDo):
    connect('DELETE FROM memorandums WHERE userID=%s AND DATE=%s AND TODO=%s;',userId,date,toDo)
    return '完成'
def day(userId):
    res=datetime.date.today().strftime('%Y/%m/%d：')
    for memo in connect('SELECT TODO FROM memorandums WHERE userID=%s AND DATE=%s ORDER BY DATE;',userId,datetime.date.today()):
            res+='\n'+memo[0]
    return res
def week(userId):
    res='到'+(datetime.date.today()+datetime.timedelta(days=6)).strftime('%m/%d')+'前：'
    for memo in connect('SELECT DATE,TODO FROM memorandums WHERE userID=%s AND DATE BETWEEN %s AND %s ORDER BY DATE;',userId,datetime.date.today(),datetime.date.today()+datetime.timedelta(days=6)):
            res+='\n'+memo[0].strftime('%Y/%m/%d：')+memo[1]
    return res
def month(userId):
    res='到'+(datetime.date.today()+datetime.timedelta(days=29)).strftime('%m/%d')+'前：'
    for memo in connect('SELECT DATE,TODO FROM memorandums WHERE userID=%s AND DATE BETWEEN %s AND %s ORDER BY DATE;',userId,datetime.date.today(),datetime.date.today()+datetime.timedelta(days=29)):
            res+='\n'+memo[0].strftime('%Y/%m/%d：')+memo[1]
    return res

#print(connect('TRUNCATE TABLE memorandums RESTART IDENTITY;'))刪除整個table並重設id從1開始
#print(connect('SELECT * FROM memorandums;'))#選整個table不限userID

抓資料成功
連線關閉
到02/06前：
2022/01/09：b一月九吃麵
2022/01/16：b一月十六吃麵


In [ ]:
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import *
import psycopg2, datetime
from psycopg2 import Error
import os

#建立變數
app = Flask(__name__)
line_bot_api = LineBotApi('')
handler = WebhookHandler('')

#處理時差問題
today=datetime.datetime.today()+datetime.timedelta(hours=8)
todayDate=today.date()

def connect(SQLcode,*variables):
    #連結資料庫
    try:
        connection = psycopg2.connect(user="",
                                      password="",
                                      host="",
                                      port="",
                                      database="")  
        cursor = connection.cursor()
        cursor.execute(SQLcode,tuple(variables))
        connection.commit()
        if cursor.description!=None:
            res=cursor.fetchall()
            print("抓資料成功")
            return res
    except (Exception, Error) as error:
        print(error)
    finally:
        if (connection):
            cursor.close()
            connection.close()
            print("連線關閉")
def allMemos(userId):
    #回傳使用者全部的排程
    res=''
    for memo in connect('SELECT * FROM memorandums WHERE userID=%s ORDER BY DATE;',userId):
            res+=memo[2].strftime('%Y/%m/%d ')+memo[3]+'\n'
    if res=="":
        return "沒有排程"
    else:
        return res
def newMemo(userId,date,toDo):
    #新增一個排程
    try:
        date=datetime.date.fromisoformat(date)
        event=connect("SELECT * FROM memorandums WHERE userID=%s AND DATE=%s AND TODO=%s",userId,date,toDo)
        if event==[] or event==None:
            connect('INSERT INTO memorandums(userID,DATE,TODO) VALUES (%s,%s,%s);',userId,date,toDo)
            return "新增成功:%s %s"%(date,toDo)
        else:
            return "錯誤:已有重複排程"
    except:
        return "格式錯誤 請確定格式是否正確 ex:新增 2022-01-01 新年快樂"
def deleteMemo(userId,date,toDo):
    #刪除一個排程
    event=""
    try:
        date=datetime.date.fromisoformat(date)
        event=connect("SELECT * FROM memorandums WHERE userID=%s AND DATE=%s AND TODO=%s",userId,date,toDo)
        if event==[] or event==None:
            return "錯誤:查無此排程"
        else:
            connect('DELETE FROM memorandums WHERE userID=%s AND DATE=%s AND TODO=%s;',userId,date,toDo)
            return "刪除成功:%s %s"%(date,toDo)
    except:
        return "格式錯誤 請確定格式是否正確 ex:刪除 2022-01-01 新年快樂"
def deleteAll(userId):
    #刪除全部排程
    event=connect("SELECT * FROM memorandums WHERE userID=%s;",userId)
    if event==[] or event==None:
        return "錯誤:沒有排程"
    else:
        event=connect('DELETE FROM memorandums WHERE userID=%s;',userId)
        return "全部排程刪除成功"
def deletePast(userId):
    #刪除過去排程
    event=connect("SELECT * FROM memorandums WHERE userID=%s AND DATE<%s;",userId,todayDate)
    if event==[] or event==None:
        return "錯誤:沒有過去排程"
    else:
        connect('DELETE FROM memorandums WHERE userID=%s AND DATE<%s;',userId,todayDate)
        return "過去排程刪除成功"
def day(userId):
    #查詢當日排程
    res=todayDate.strftime('%Y/%m/%d：')
    event=""
    for memo in connect('SELECT TODO FROM memorandums WHERE userID=%s AND DATE=%s ORDER BY DATE;',userId,todayDate):
            event+='\n'+memo[0]
    if event=="":
        return "今日沒有排程"
    else:
        return res+event
def week(userId):
    #查詢當周排程
    res='到'+(todayDate+datetime.timedelta(days=6)).strftime('%m/%d')+'前：'
    event=""
    for memo in connect('SELECT DATE,TODO FROM memorandums WHERE userID=%s AND DATE BETWEEN %s AND %s ORDER BY DATE;',userId,todayDate,todayDate+datetime.timedelta(days=6)):
            event+='\n'+memo[0].strftime('%Y/%m/%d：')+memo[1]
    if event=="":
        return "本周沒有排程"
    else:
        return res+event
def month(userId):
    #查詢當月排程
    res='到'+(todayDate+datetime.timedelta(days=29)).strftime('%m/%d')+'前：'
    event=""
    for memo in connect('SELECT DATE,TODO FROM memorandums WHERE userID=%s AND DATE BETWEEN %s AND %s ORDER BY DATE;',userId,todayDate,todayDate+datetime.timedelta(days=29)):
            event+='\n'+memo[0].strftime('%Y/%m/%d：')+memo[1]
    if event=="":
        return "本月沒有排程"
    else:
        return res+event
def help():
    #說明書，回傳所有指令
    return "新增:新增一個排程\n  ex:新增 2022-01-01 新年快樂\n移除:移除一個排程\n  ex:移除 2022-01-01 新年快樂\n當日:查詢當日排程\n當周:查詢7天內排程\n當月:查詢30天內排程\n全部:查詢全部排程\n全部清除:清除全部排程\n清除歷程:清除過去排程"

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
 
  
    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
 
    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
 
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    mes = event.message.text
    #傳給機器人的訊息
    userID = event.source.user_id
    #獲取使用者的ID建立資料
    if mes=="help":
        botreply= TextSendMessage(text=help())
    elif "新增" in mes:
        try:
            com,date,toDo=mes.split()
            botreply= TextSendMessage(text=newMemo(userID,date,toDo))
        except:
            botreply= TextSendMessage(text="格式錯誤 請確定格式是否正確 ex:新增 2022-01-01 新年快樂")
    elif "移除" in mes:
        try:
            com,date,toDo=mes.split()
            botreply= TextSendMessage(text=deleteMemo(userID,date,toDo))
        except:
            botreply= TextSendMessage(text="格式錯誤 請確定格式是否正確 ex:移除 2022-01-01 新年快樂")
    elif mes=="當日":
        botreply= TextSendMessage(text=day(userID))
    elif mes=="當周":
        botreply= TextSendMessage(text=week(userID))
    elif mes=="當月":
        botreply= TextSendMessage(text=month(userID))
    elif mes=="全部":
        botreply= TextSendMessage(text=allMemos(userID))
    elif mes=="全部清除":
        botreply= TextSendMessage(text=deleteAll(userID))
    elif mes=="清除歷程":
        botreply= TextSendMessage(text=deletePast(userID))
    else :
        botreply= TextSendMessage(text="格式錯誤，請輸入help獲得幫助")
    line_bot_api.reply_message(event.reply_token,botreply)
    #根據使用者的指令決定機器人的回覆

if __name__ == "__main__":
    port = int(os.environ.get('PORT', 5000))
    app.run(host='0.0.0.0', port=port)

In [ ]:
from flask import Flask, request, abort
 
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import *

import psycopg2

app = Flask(__name__)
 
line_bot_api = LineBotApi('')

handler = WebhookHandler('02bcc849df5cc9e1729537bd1d97f769')

line_bot_api.push_message('Ua8898e3c489c5b69e61b4f53cb230410', TextSendMessage(text='程式已啟動'))

class PostgresBaseManager:

    def __init__(self):

        self.database = ''
        self.user = ''
        self.password = ''
        self.host = ''
        self.port = '5432'
        self.conn = self.connectServerPostgresDb()

    def connectServerPostgresDb(self):
        """
        :return: 連接 Heroku Postgres SQL 認證用
        """
        conn = psycopg2.connect(
            database=self.database,
            user=self.user,
            password=self.password,
            host=self.host,
            port=self.port)
        return conn
    
    def createTable(self):
        # create a table
        create_table_query = '''
            CREATE TABLE memo(
            id     serial  PRIMARY KEY,
            date   DATE    NOT NULL,
            event  VARCHAR NOT NULL);
            '''

        # execute query
        cur = self.conn.cursor()
        cur.execute(create_table_query)
        self.conn.commit()

        cur.close()
    def insertTable(self,date,event):
        records = [
            (date, event)
        ]
        
        cur = self.conn.cursor()
        
        # table_columns = '(date,event)'
        # postgres_insert_query = f"""INSERT INTO memo {table_columns} VALUES (%s,%s)"""
        # cur = self.conn.cursor()
        # cur.executemany(postgres_insert_query, records)
        # self.conn.commit()

        records = (date, event)
        table_columns = '(date,event)'
        postgres_insert_query = f"""INSERT INTO memo {table_columns} VALUES (%s,%s)"""

        cur.execute(postgres_insert_query, records)
        self.conn.commit()

    def deleteTable(self):
        postgres_delete_query = f"""DELETE FROM memo WHERE id > 5"""
        
        cur = self.conn.cursor()
        cur.execute(postgres_delete_query)
        self.conn.commit()

        count = cur.rowcount
        print(count, "Record delete successfully into database")

        cur.close()
    def getTable(self):
        postgres_select_query = f"""SELECT * FROM memo"""
        # f"""SELECT event FROM memo WHERE date = 今日日期"""
        cur = self.conn.cursor()

        cur.execute(postgres_select_query)
        data = cur.fetchall()
        print(data)
        cur.close()
        
        return str(data) 
    def closePostgresConnection(self):
        """
        :return: 關閉資料庫連線使用
        """
        self.conn.close()

    def runServerPostgresDb(self):
        """
        :return: 測試是否可以連線到 Heroku Postgres SQL
        """
        cur = self.conn.cursor()
        cur.execute('SELECT VERSION()')
        results = cur.fetchall()
        print("Database version : {0} ".format(results))
        self.conn.commit()
        cur.close()

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
 
  
    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
 
    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
 
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    mes = event.message.text
    #傳給機器人的訊息
    if mes=="help":
        try:
            botreply= TextSendMessage(text="這是罐頭訊息")
        except:
            botreply= TextSendMessage(text="格式錯誤 請輸入 help ")
    elif "新增" in mes:
        try:
            com,date,event=mes.split()
            postgres_manager.insertTable(date,event)
            botreply= TextSendMessage(text="新增成功")
        except:
            botreply= TextSendMessage(text="格式錯誤 請確定格式是否正確 ex:新增 2022-01-01 新年快樂")
    elif "移除" in mes:
        try:
            #postgres_manager.deleteTable()
            #botreply= TextSendMessage(text="移除成功")
            botreply= TextSendMessage(text="test")
        except:
            botreply= TextSendMessage(text="格式錯誤 請確定格式是否正確 ex:移除 2022-01-01 新年快樂")
    elif mes=="當日":
        botreply= TextSendMessage(text=postgres_manager.getTable())
    elif mes=="當周":
        botreply= TextSendMessage(text=postgres_manager.getTable())
    elif mes=="當月":
        botreply= TextSendMessage(text=postgres_manager.getTable())
    else :
        botreply= TextSendMessage(text="格式錯誤 請輸入 help ")

    #機器人回復
    line_bot_api.reply_message(event.reply_token,botreply)

import os
if __name__ == "__main__":
    postgres_manager = PostgresBaseManager()
    port = int(os.environ.get('PORT', 5000))
    app.run(host='0.0.0.0', port=port)

In [1]:
from tkinter import *
from tkcalendar import Calendar
import pickle
memos=[]
tempEntries=[]
try:
    with open('memos.pkl', 'rb') as f:
        memos = pickle.load(f)
except (FileNotFoundError,EOFError):
    with open('memos.pkl', 'wb') as f:
        pickle.dump(memos, f)
    with open('memos.pkl', 'rb') as f:
        memos = pickle.load(f)       
def day():
    print('t')
def week():
    print('t')
def month():
    print('t')
def newMemo(winToClose):
    aMemo=LabelFrame(window,text=newDate.cal.get_date())
    aMemo.pack()
    entry=Entry(aMemo)
    tempEntries.append(entry)
    entry.pack()
    memos.append([newDate.cal.get_date(),''])
    winToClose.destroy()
def newDate():
    datePicker=Toplevel(window)
    datePicker.title('選擇日期')
    datePicker.geometry('400x400')
    newDate.cal = Calendar(datePicker, selectmode = 'day')
    newDate.cal.pack(pady = 20)
    Button(datePicker, text = "OK", command = lambda: newMemo(datePicker)).pack(pady = 20)
def save():
    i=0
    for entry in tempEntries:
        memos[i][1]=entry.get()
        i+=1
    with open('memos.pkl', 'wb') as f:
        pickle.dump(memos, f)
    done=Label(window,text='done')
    done.place(x=330,y=553)
    done.after(1000, lambda: done.destroy() )
window=Tk()
window.title('備忘錄')
window.geometry('600x600')
menus=Menu(window)
window.config(menu=menus)
menus.add_command(label='新增一筆',command=newDate)
menus.add_command(label='本日排程',command=day)
menus.add_command(label='本週排程',command=week)
menus.add_command(label='本月排程',command=month)
for memo in memos:
    aMemo=LabelFrame(window,text=memo[0])
    aMemo.pack()
    entry=Entry(aMemo)
    entry.insert(0,memo[1])
    tempEntries.append(entry)
    entry.pack()
Button(window,text='save',command=save).place(x=290,y=550)
mainloop()

In [2]:
import datetime
print(datetime.datetime.today())

2022-01-09 05:01:13.767878
